# Skywalker algorithm against the lcls mirror systems

In [ ]:
SIMULATION = False
#ALIGNMENT = 'HOMS'
#ALIGNMENT = 'MEC'
ALIGNMENT = 'MFX'
#ALIGNMENT = 'XCS'

alignment_goals = dict(HOMS=[210, 270],
                       MEC=[270],
                       MFX=[268],
                       XCS=[270, 270])
first_steps = 5 # Naive first step for the parameters search
tolerances = 10 # Tolerance of each target in Pixels
average = 100 # Number of shots to average over
log_level = "INFO"

# PARAMETERS FOR SIMULATION ONLY
centroid_noise = 5.0 # Noise level of centroid measurements
infinite_yag = False # Assume all yags are infinitely large

In [ ]:
############
# Standard #
############
import os
import sys
import logging
import argparse

###############
# Third Party #
###############
from bluesky import RunEngine
from bluesky.tests.utils import MsgCollector
from bluesky.plans import run_wrapper
from pcdsdevices.sim import pim, source, mirror

In [ ]:
"""
Parameters
----------
"""
#Configure logger
log_level = getattr(logging, log_level, None)

#Report invalid logging level
if not isinstance(log_level, int):
    raise ValueError("Invalid log level : {}".format(log_level))

#Create basic configuration
logging.basicConfig(level=log_level, format='%(name)s - %(message)s')

from pswalker.examples  import patch_pims
from pswalker.skywalker import (skywalker, homs_system)
from pswalker.plans import walk_to_pixel
from pswalker.watcher import Watcher
#Instantiate simulation
if not SIMULATION:
    print("*"*80)
    print("* WARNING: Running on real devices in Production...")
    print("*"*80)
    input("Press any key to continue...")
    system = homs_system()
    m1h = system['m1h']
    m1h2 = system['m1h2']
    m2h = system['m2h']
    m2h2 = system['m2h2']
    xrtm2 = system['xrtm2']
    xrtm22 = system['xrtm22']
    hx2 = system['hx2']
    dg3 = system['dg3']
    mfxdg1 = system['mfxdg1']
    mecy1 = system['mecy1']
    m1 = m1h
    m2 = m2h
    y1 = hx2
    y2 = dg3
    
    xrtm2.settle_time = 10
else:
    s = source.Undulator('test_undulator')
    m1 = mirror.OffsetMirror('test_m1h', 'test_m1h_xy',
                              z=90.510, alpha=0.0014)
    m2 = mirror.OffsetMirror('test_m2h', 'test_m2h_xy',
                              x=0.0317324, z=101.843, alpha=0.0014)
    xrtm2 = mirror.OffsetMirror('test_xrtm2', 'test_xrtm2_xy',
                                 x=0.0317324, z=200, alpha=0.0014)
    y1 = pim.PIM('test_p3h', x=0.0317324, z=103.660,
             zero_outside_yag= not infinite_yag)
    y2 = pim.PIM('test_dg3', x=0.0317324, z=375.000,
             zero_outside_yag= not infinite_yag)
    mecy1 = pim.PIM('test_mecy1', x=0.0317324, z=350,
             zero_outside_yag= not infinite_yag)
    mfxdg1 = mecy1
    patch_pims([y1, y2], mirrors=[m1, m2], source=s)
    patch_pims([mecy1], mirrors=[xrtm2], source=s)

    #Add noise
    y1.centroid_noise = centroid_noise
    y2.centroid_noise = centroid_noise
    mecy1.centroid_noise = centroid_noise

alignment_yags = dict(HOMS=[y1, y2],
                      MEC=[mecy1],
                      MFX=[mfxdg1],
                      XCS=['pbty', 'xcssb1y'])
alignment_mots = dict(HOMS=[m1, m2],
                      MEC=[xrtm2],
                      MFX=[xrtm2],
                      XCS=['xrtm1', 'xrtm3'])

In [ ]:
#Create Skywalker plan
yags = alignment_yags[ALIGNMENT]
mots = alignment_mots[ALIGNMENT]
goals = alignment_goals[ALIGNMENT]
if SIMULATION:
    det_rbv = 'detector_stats2_centroid_x'
else:
    det_rbv = 'detector_stats2_centroid_y'
mot_rbv = 'pitch'
plan = skywalker(yags, mots, det_rbv, mot_rbv,
        goals, first_steps=first_steps, tolerances=tolerances,
        averages=average, timeout=10)  

In [ ]:
#Create RunEngine
RE = RunEngine({})
RE.record_interruptions = True
#Subscribe a Watcher instance
watcher = Watcher()
RE.msg_hook = watcher
RE.subscribe('all',  watcher)


In [ ]:
RE(plan)

In [ ]:
#Print Run Statistics
report = watcher.report()